This notebook is used to do all the evaluations on the results of the different methods using hard majority labels.

In [ ]:
import numpy as np
import pandas as pd
import csv
from statsmodels.stats import inter_rater as irr
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

# The methods need to be compared to the annotated data, for which we need to get the proper array
# read the annotated files to dataframes
annotated1 = pd.read_csv('annotatedData/Emotion-1.csv', sep=';', header=0)
annotated2 = pd.read_csv('annotatedData/Emotion-2.csv', sep=';', header=0)
annotated3 = pd.read_csv('annotatedData/Emotion-3.csv', sep=';', header=0)
annotated4 = pd.read_csv('annotatedData/Emotion-4.csv', sep=',', header=0)
annotated5 = pd.read_csv('annotatedData/Emotion-5.csv', sep=',', header=0)

# drop the not needed columns. We care only about the labels, not the text.
dropList = ["id", "question_id", "participant_id", "frisian", "dutch", "english"]
annotated1Clean = annotated1.drop(dropList, axis=1)
annotated2Clean = annotated2.drop(dropList, axis=1)
annotated3Clean = annotated3.drop(dropList, axis=1)
annotated4Clean = annotated4.drop(dropList, axis=1)
annotated5Clean = annotated5.drop(dropList, axis=1)
display(annotated1Clean)
# combine the data frames
annotatedDataFrame12 = annotated1Clean.add(annotated2Clean, fill_value=0)
annotatedDataFrame34 = annotated3Clean.add(annotated4Clean, fill_value=0)
annotatedDataFrame345 = annotatedDataFrame34.add(annotated5Clean, fill_value=0)
annotatedDataFrame = annotatedDataFrame345.add(annotatedDataFrame12, fill_value=0)
# replace the NaN with 0
annotatedDataFrameNoNaN = annotatedDataFrame.fillna(0)
# transform the annotated data to a numpy array to do the evaluations with
numAnnotatedSummed = pd.DataFrame.to_numpy(annotatedDataFrameNoNaN.iloc[0:50])
# Then a threshold is set on which labels are considered "true"
numAnnotated = np.where(numAnnotatedSummed < 1, 0, 1)
print(numAnnotated)
labelsToAnnotate: str = "admiration;amusement;anger;annoyance;approval;caring;confusion;curiosity;desire;disappointment;disapproval;disgust;embarrassment;excitement;fear;gratitude;grief;joy;love;nervousness;optimism;pride;realization;relief;remorse;sadness;suprise;neutral"
labelsList = labelsToAnnotate.split(";")
# First the methods needed for the evaluation
# This method can be applied to a dataframe to get the actual labels instead of just numbers
def getLabels(data):
    result = ""
    labelsToAnnotate: str = "admiration;amusement;anger;annoyance;approval;caring;confusion;curiosity;desire;disappointment;disapproval;disgust;embarrassment;excitement;fear;gratitude;grief;joy;love;nervousness;optimism;pride;realization;relief;remorse;sadness;suprise;neutral"
    labelsList = labelsToAnnotate.split(";")
    for label in labelsList:
        if data[label] >= 1:
            result = result + ", " + label
    return result

# Gets the number of unique labels
def getNumLabels(resultArray, runs):
    amountLabels = []
    for n in range(1, runs+1):
        amountLabels.append("label " + str(n) + ": " + str(np.count_nonzero(resultArray >= n)))
    return amountLabels

# Calculates the fleiss kappa score of an array
def calculateAgreement(resultArray):
    agg = irr.aggregate_raters(resultArray) # returns a tuple (data, categories)
    resultOneShot = irr.fleiss_kappa(agg[0], method='fleiss')
    return resultOneShot

# Calculates the micor f1, recall and precision
def calcF1RecallPrecision(annoArray, predictArray, runs):
    eval = pd.DataFrame(columns=["F1 score", "Recall", "Precision"])
    for n in range (1, runs+1):
        predictArrayThres = np.where(predictArray < n, 0, 1)
        f1Score = f1_score(annoArray, predictArrayThres, average="micro")
        recall = recall_score(annoArray, predictArrayThres, average="micro")
        precision = precision_score(annoArray, predictArrayThres, average="micro")
        eval.loc[n] = [f1Score, recall, precision]
    return eval


Zeroshot

In [ ]:
arrayZeroShotNull = np.array([[ 1,  1,  4,  0,  1,  1,  2,  0,  0,  2,  3,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  0,  0,  4,  4,  6,  2,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6],
       [ 1,  0,  2,  0,  4,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5],
       [ 3,  0,  0,  0,  7,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 7,  0,  0,  0,  7,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2],
       [ 0,  0,  3,  0,  0,  0,  1,  0,  0,  0,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6],
       [ 0,  0, 10,  0,  3,  0,  0,  0,  0,  0,  5,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3],
       [ 0,  0,  9,  1,  0,  0,  0,  0,  0,  0,  7,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4],
       [ 9,  0,  0,  0, 10,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [10,  0,  0,  0, 10,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,
         0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9],
       [ 0,  0,  9,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  1,  0,  8,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3],
       [ 0,  0,  0,  0,  5,  2,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3],
       [ 1,  0,  2,  0,  3,  0,  0,  1,  0,  0,  2,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5],
       [ 0,  0,  1,  0,  2,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9],
       [ 0,  0,  0,  0,  6,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3],
       [ 9,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  9,  1,  0,  0,  0,  0,  0,  0,  8,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 3,  0,  3,  0,  3,  0,  4,  2,  0,  1,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  7],
       [ 3,  0,  6,  0,  2,  0,  0,  0,  0,  0,  7,  0,  0,  1,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  1,  0,  8,  1,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  6,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2],
       [ 0,  0, 10,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 3,  0,  1,  0,  5,  2,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  7],
       [ 0,  0,  1,  0,  5,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  8,  3,  0,  0,  0,  0,  0,  0, 10,  2,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2],
       [ 0,  0,  8,  0,  0,  0,  0,  0,  0,  1,  5,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 7,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 7,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  3,  0,  7,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  2],
       [ 1,  0,  0,  0,  4,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5],
       [ 5,  0,  0,  0,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  8,  0,  1,  1,  0,  0,  0,  0,  8,  1,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 9,  0,  0,  0,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 1,  0,  1,  0,  8,  7,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  7,  3,  0,  0,  1,  0,  0,  2,  4,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 1,  0,  2,  0,  3,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  2],
       [ 8,  0,  0,  0,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3],
       [ 5,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  4],
       [ 0,  0,  9,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3],
       [ 4,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 5,  0,  0, 10,  3,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  2,  7,  0,  0,  2,  3,  0,  6,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2],
       [ 4,  0,  0,  0,  9,  1,  0,  2,  2,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6],
       [ 3,  0,  0,  0,  7,  2,  0,  0,  0,  2,  3,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  3],
       [ 0,  0,  7,  0,  2,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6],
       [ 1,  0,  1,  0,  0,  0,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9],
       [ 9,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1]])
# Now we calculate the different scores
predictArray = arrayZeroShotNull
kappa = calculateAgreement(predictArray)
print("Kappa score: " + str(kappa))

f1RePr = calcF1RecallPrecision(numAnnotated, predictArray, 10)
display(f1RePr)

num = getNumLabels(predictArray, 10)
print(num)

frameZeroShot = pd.DataFrame(predictArray, columns= labelsList)
labels = frameZeroShot.apply(getLabels, axis=1)
pd.set_option('display.max_colwidth', None)
display(labels)

Oneshot

In [ ]:
arrayOneShotNull = np.array([[ 0,  0,  0,  8,  0,  3,  4,  3,  1,  5,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  0,  0,  2,  4,  4, 10,  2,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  2,  0,  1,  0,  0,  0,  0,  4],
       [ 0,  0,  0,  0,  7,  0,  0,  5,  3,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  3],
       [ 0,  0,  0,  0, 10,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  4,  2,  0,  1,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  7],
       [ 0,  0,  0,  0, 10,  9,  0,  0,  6,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 10,  9,  0,  0,  0,  0,  0,  0, 10,  1,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  9,  0,  0,  4,  3,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  1],
       [ 1,  0,  0,  0, 10,  4,  0,  2,  7,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 10,  4,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  6,  0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  1, 10,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  8],
       [ 0,  0,  0,  0, 10, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  9,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  8,  1,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  9],
       [ 0,  0,  0,  0, 10,  4,  0,  1,  5,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  7,  0,  0,  0,  0,  0,  0,  2],
       [ 0,  0,  0,  0, 10,  4,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  8,  0,  0,  0,  0,  0,  0,  3],
       [ 0,  0,  0,  0,  6,  8,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  9,  0,  0,  0,  0,  0,  0,  2],
       [ 5,  0,  0,  0, 10,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  6,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  9,  7,  0,  0,  0,  0,  0,  1, 10,  1,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  3,  2,  5,  5,  1,  3,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  6],
       [ 0,  0,  2,  0,  8,  0,  0,  0,  7,  1,  7,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  9,  6,  0,  0,  6,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  7,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  8,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4],
       [ 0,  0, 10,  3,  0,  0,  0,  0,  0,  0, 10,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 10,  1,  0,  1,  8,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  6,  0,  0,  0,  0,  0,  0,  3],
       [ 0,  0,  0,  0, 10,  1,  0,  1,  7,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  6,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0, 10,  9,  0,  0,  7,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  4,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  7,  3,  0,  0,  0,  1,  0,  0, 10,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8],
       [ 0,  0,  8,  2,  0,  0,  0,  0,  0,  3,  9,  0,  0,  0,  1,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 2,  0,  0,  0, 10,  2,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 10,  2,  0,  0,  6,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  8,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 10,  5,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  8,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 10, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 10,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  9,  8,  0,  0,  1,  1,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  7],
       [ 1,  0,  0,  0, 10,  9,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  7,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  9, 10,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  2,  2,  3,  1,  4,  0,  4,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9],
       [ 0,  0,  0,  0, 10,  1,  0,  0,  7,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  0],
       [ 6,  0,  0,  0, 10,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  7,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  8,  0,  0,  0,  0,  0,  0,  2],
       [ 0,  0,  0,  0,  4,  1,  1,  0,  1,  2,  4,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8],
       [ 0,  0,  0,  0, 10,  1,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0, 10,  3,  0,  0,  0,  0,  1,  6,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  5],
       [ 0,  0,  0,  0,  0,  4,  1,  6,  8,  2,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  6],
       [ 0,  0,  0,  0, 10,  2,  0,  1,  7,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  9,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  7,  5,  0,  0,  2,  3,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  4,  0,  0,  0,  0,  0,  0,  7],
       [ 0,  0,  0,  0,  9,  6,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  6,  0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  0,  0,  0,  1,  1,  5,  1,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  2,  1,  0,  0,  0,  0,  0,  8],
       [ 5,  0,  0,  0,  6,  0,  0,  0,  8,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  9,  0,  0,  0,  0,  0,  0,  0]])
# Now we calculate the different scores
predictArray = arrayOneShotNull
kappa = calculateAgreement(predictArray)
print("Kappa score: " + str(kappa))

# as row 4 is used to give examples, it is removed for the evaluation
numAnno = np.delete(numAnnotated, 4, axis=0)
f1RePr = calcF1RecallPrecision(numAnno, predictArray, 10)
display(f1RePr)

num = getNumLabels(predictArray, 10)
print(num)

frameZeroShot = pd.DataFrame(predictArray, columns= labelsList)
labels = frameZeroShot.apply(getLabels, axis=1)
pd.set_option('display.max_colwidth', None)
display(labels)

Fewshot

In [ ]:
arrayFewShotNull = np.array([[ 0,  0,  0,  2,  0,  2,  4,  1,  0,  1,  4,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8],
       [ 0,  0,  0,  0,  5,  8,  9,  1,  1,  1,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6],
       [ 0,  0,  0,  0,  9,  0,  0,  6,  1,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0, 10,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10],
       [ 0,  0, 10,  4,  0,  0,  0,  0,  0,  0, 10,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 10,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  8,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0, 10,  4,  0,  2,  2,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 10,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  6],
       [ 0,  0,  0,  5,  3,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5],
       [ 0,  0,  0,  0, 10, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2],
       [ 0,  0,  0,  0, 10,  3,  0,  1,  2,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  0,  0, 10,  2,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  9,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 10,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9],
       [ 0,  0,  0,  0, 10,  7,  0,  2,  1,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0],
       [ 2,  0,  0,  0, 10,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  4,  6,  0,  0,  0,  0,  0,  0, 10,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  2,  2,  0,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9],
       [ 0,  0,  5,  2,  3,  0,  0,  0,  1,  0,  9,  1,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8],
       [ 0,  0,  8,  3,  0,  0,  0,  0,  0,  0, 10,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 10,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5],
       [ 1,  0,  1,  0, 10,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0, 10,  8,  0,  0,  7,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  6,  9,  0,  0,  0,  0,  0,  0,  9,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2],
       [ 0,  0,  9,  0,  0,  0,  0,  0,  0,  0,  9,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2],
       [ 1,  0,  0,  0,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  7,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  3,  1,  0,  0,  0,  0,  0,  3],
       [ 0,  0,  0,  0, 10,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  0,  0, 10, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,
         0,  0,  0,  0,  9,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  6,  6,  0,  0,  0,  1,  3,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  7],
       [ 0,  0,  0,  0, 10, 10,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  4,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 10, 10,  0,  0,  0,  2,  2,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  2],
       [ 0,  0,  0,  0,  0,  2,  0,  0,  0,  3,  3,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10],
       [ 0,  0,  0,  0, 10,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  8,  0,  0,  0,  0,  0,  0,  0],
       [ 8,  0,  0,  0, 10,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  9,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3],
       [ 0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  9,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 10,  0,  0,  0,  0,  0,  0, 10,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3],
       [ 0,  0,  0,  5,  0,  0,  0,  0,  0,  2,  2,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  7],
       [ 0,  0,  0,  0, 10,  0,  0,  1,  3,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  6,  0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  0,  1,  9,  6,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  9],
       [ 0,  0,  0,  0, 10,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8],
       [ 0,  0,  0,  0,  0,  1,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10],
       [ 1,  0,  0,  0, 10,  0,  0,  0, 10,  0,  0,  0,  0,  1,  0,  0,
         0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0]])
# Now we calculate the different scores
predictArray = arrayFewShotNull
kappa = calculateAgreement(predictArray)
print("Kappa score: " + str(kappa))

# as row 4, 6 and 21 are used to give examples, those are removed for the F1 score.
numAnno = np.delete(numAnnotated, [4, 6, 21], axis=0)
f1RePr = calcF1RecallPrecision(numAnno, predictArray, 10)
display(f1RePr)

num = getNumLabels(predictArray, 10)
print(num)

frameZeroShot = pd.DataFrame(predictArray, columns= labelsList)
labels = frameZeroShot.apply(getLabels, axis=1)
pd.set_option('display.max_colwidth', None)
display(labels)

ZeroshotCoT

In [ ]:
arrayZeroShotCoTNull = np.array([[ 0,  0,  1,  0,  1,  5,  7,  1,  0,  2,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8],
       [ 0,  0,  0,  0,  1,  2,  3,  5,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8],
       [ 0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  1,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  6,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5],
       [ 0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3],
       [ 0,  0,  2,  2,  0,  0,  0,  0,  0,  1,  9,  1,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2],
       [ 0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2],
       [ 1,  0,  0,  0,  4,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,
         0,  0,  0,  0,  9,  0,  0,  0,  0,  0,  0,  3],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10],
       [ 0,  0,  1,  1,  0,  0,  0,  0,  0,  0, 10,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3],
       [ 0,  0,  0,  0,  2,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  1,  1,  4,  1,  0,  0,  0,  3,  4,  0,  0,  0,  0,  1,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  4,  0,  0,  0,  0,  0,  0,  3],
       [ 0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  2],
       [ 0,  0,  3,  4,  0,  0,  0,  0,  0,  0,  8,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  4,  2,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5],
       [ 0,  0,  9,  0,  1,  0,  0,  0,  1,  0,  4,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  1,  0,  1,  0,  0,  0,  2,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3],
       [ 0,  0,  2,  6,  0,  0,  0,  0,  0,  0,  6,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  9],
       [ 0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  7],
       [ 0,  0,  0,  0,  2,  5,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  3,  4,  7,  2,  0,  0,  0,  0,  1,  8,  3,  0,  0,  1,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  1,  1,  0,  0,  0,  0,  0,  0, 9,  0,  0,  0,  1,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  2,  0,  0,  0,  1,  0,  2,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  5],
       [ 0,  0,  0,  0,  1,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  7],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  4,  3,  0,  0,  0,  1,  0,  4,  5,  1,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 3,  0,  0,  0,  2,  2,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  2,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3],
       [ 0,  0,  0,  0,  0,  0,  1,  2,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8],
       [ 0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  2],
       [ 2,  0,  0,  0,  3,  1,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9],
       [ 0,  0,  6,  1,  0,  0,  1,  1,  0,  1,  3,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  2],
       [ 0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  9,  1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6],
       [ 0,  0,  1,  7,  0,  0,  3,  1,  3,  3,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  5,  3,  0,  3,  1,  0,  2,  1,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  2],
       [ 0,  0,  1,  0,  1,  4,  0,  0,  0,  0,  8,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3],
       [ 0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9],
       [ 0,  0,  0,  0,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8],
       [ 2,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])
# Now we calculate the different scores
predictArray = arrayZeroShotCoTNull
kappa = calculateAgreement(predictArray)
print("Kappa score: " + str(kappa))

f1RePr = calcF1RecallPrecision(numAnnotated, predictArray, 10)
display(f1RePr)

num = getNumLabels(predictArray, 10)
print(num)

frameZeroShot = pd.DataFrame(predictArray, columns= labelsList)
labels = frameZeroShot.apply(getLabels, axis=1)
pd.set_option('display.max_colwidth', None)
display(labels)

OneshotCoT

In [ ]:
arrayOneShotCoTNull = np.array([[ 0,  4,  0,  4,  1,  1,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10],
       [ 0,  0,  0,  0,  0,  1,  8,  4,  0,  0,  1,  0,  0,  0,  4,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0, 10],
       [ 0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10],
       [ 0,  0,  0,  0,  1,  0,  0,  0,  0,  0, 10,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  1,  0,  0,  0,  0,  0,  6],
       [ 0,  0,  7,  1,  0,  0,  0,  0,  0,  3,  9,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  1,  9,  0,  1,  0,  0,  1,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  8,  0,  1,  0,  0,  0,  0,  7],
       [ 0,  0,  0,  0,  0,  4,  0,  0,  3,  0,  1,  0,  0,  1,  0,  7,
         0,  0,  0,  0,  9,  3,  0,  0,  0,  0,  0,  2],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10,  0,  0,  0,  1,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 10,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  0,  0,  4,  0,  2,  0,  0,  0,  4,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  6],
       [ 0,  0,  1,  5,  5,  1,  0,  0,  0,  5,  3,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10],
       [ 0,  0,  0,  0,  1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  9],
       [ 0,  0,  0,  0,  6,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  1,  0,  0,  0,  0,  0,  3],
       [ 0,  0,  8,  6,  0,  0,  0,  0,  0,  1, 10,  2,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  1,  7,  1,  1,  0,  0,  0,  0,  0,  0,  0,  4,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  4,  2,  3,  0,  0,  0,  3,  1,  7,  1,  0,  1,  1,  0,
         0,  0,  0,  1,  5,  1,  0,  0,  0,  0,  0,  3],
       [ 0,  0,  1,  1,  4,  1,  0,  0,  0,  0,  5,  0,  0,  0,  1,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  8],
       [ 0,  0,  9,  4,  0,  0,  0,  0,  0,  0,  9,  0,  0,  0,  1,  0,
         0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  3],
       [ 0,  0,  0,  1,  5,  1,  0,  0,  4,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  4,  0,  0,  0,  0,  0,  0,  6],
       [ 0,  0,  0,  0,  3,  0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  9,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  5,  8,  0,  0,  0,  0,  0,  0,  9,  2,  0,  0,  2,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  2,  2,  0,  0,  0,  0,  0,  1, 10,  2,  0,  0,  9,  0,
         0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  0,  0,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  5],
       [ 0,  0,  0,  0,  7,  1,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  6,  1,  0,  0,  0,  0,  0,  2],
       [ 0,  0,  0,  0,  6,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  6],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10],
       [ 0,  0,  0,  0,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  9,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  3,  0,  0,  0,  3,  5,  1,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8],
       [ 0,  0,  0,  0,  3,  5,  0,  0,  6,  0,  0,  0,  0,  2,  0,  1,
         0,  4,  0,  0,  9,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  3,  4,  0,  0,  1,  0,  0,  0,  0,  0,  1,  0,
         0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  6],
       [ 0,  0,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  8],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  7,  1,  0,  0,  0,  0,  0,  4],
       [ 1,  0,  0,  0,  7,  2,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  5,  1,  0,  0,  0,  0,  0,  7],
       [ 0,  0,  0,  0,  1,  1,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10],
       [ 0,  0,  1,  5,  0,  0,  0,  0,  0,  0,  6,  2,  0,  0,  5,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  4,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  9,  0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  0, 10,  3,  0,  0,  0,  0,  1,  2,  0,  0,  0,  1,  0,
         0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  5],
       [ 0,  0,  0,  5,  0,  0,  1,  0,  5,  3,  0,  0,  0,  1,  0,  0,
         0,  0,  0,  0,  1,  0,  2,  0,  0,  0,  0,  5],
       [ 0,  0,  0,  0,  3,  4,  0,  0,  8,  0,  2,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  4,  0,  0,  0,  0,  0,  0,  6],
       [ 0,  0,  0,  1,  1,  8,  0,  0,  0,  0,  9,  0,  0,  0,  0,  1,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  6],
       [ 0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10],
       [ 0,  0,  0,  0,  2,  4,  4,  1,  0,  2,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  8],
       [ 0,  0,  0,  0,  1,  0,  0,  0,  7,  0,  0,  0,  0,  5,  0,  0,
         0,  0,  0,  0, 10,  1,  0,  0,  0,  0,  0,  0]])
# Now we calculate the different scores
predictArray = arrayOneShotCoTNull
kappa = calculateAgreement(predictArray)
print("Kappa score: " + str(kappa))

# as row 4 is used to give examples, it is removed for the evaluation
numAnno = np.delete(numAnnotated, 4, axis=0)
f1RePr = calcF1RecallPrecision(numAnno, predictArray, 10)
display(f1RePr)

num = getNumLabels(predictArray, 10)
print(num)

frameZeroShot = pd.DataFrame(predictArray, columns= labelsList)
labels = frameZeroShot.apply(getLabels, axis=1)
pd.set_option('display.max_colwidth', None)
display(labels)

FewshotCoT

In [ ]:
arrayFewShotCoTNull = np.array([[ 0,  1,  0,  5,  4,  0,  0,  1,  0,  0,  1,  0,  0,  2,  1,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9],
       [ 0,  0,  0,  3,  3,  0,  3,  3,  0,  1,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  8],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8],
       [ 0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10],
       [ 0,  0,  1,  8,  0,  0,  0,  0,  0,  0, 10,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  1,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  6],
       [ 0,  0,  0,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6,
         0,  0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10],
       [ 0,  0,  1,  1,  0,  0,  0,  0,  0,  0, 10,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2],
       [ 0,  0,  0,  0,  3,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  7],
       [ 0,  0,  0,  2,  7,  0,  0,  0,  0,  1,  7,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6],
       [ 0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  9,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  4,  5,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  7,  9,  0,  0,  0,  0,  0,  0, 10,  3,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  4,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8],
       [ 0,  0,  1,  6,  7,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  5,  4,  0,  0,  0,  0,  0,  0,  8,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  6,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  2],
       [ 0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  0,  0,  2,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  6,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  5, 10,  0,  1,  0,  0,  0,  0, 10,  1,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  2,  3,  0,  0,  0,  0,  0,  0,  9,  0,  0,  0,  4,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  4,  0,  0,  0,  0,  0,  0,  2],
       [ 0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  1,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  5],
       [ 0,  0,  0,  0,  2,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  6],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  1,  7,  0,  0,  0,  0,  3,  1,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  7],
       [ 0,  0,  0,  0,  4,  6,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,
         0,  3,  2,  0,  8,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  2,  4,  4,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  5],
       [ 0,  0,  0,  1,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  9],
       [ 0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  7,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  1,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  3,  1,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  0,  0,  2,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  5],
       [ 0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  4,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2],
       [ 0,  0,  0,  0,  2,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  7,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 10,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9],
       [ 0,  0,  0,  6,  3,  0,  0,  0,  4,  3,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  5],
       [ 0,  0,  0,  0,  9,  0,  0,  0,  2,  0,  3,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  6],
       [ 0,  0,  0,  4,  1,  5,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  7],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9],
       [ 0,  0,  0,  0,  0,  1,  3,  1,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  9],
       [ 0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  4,  0,  0,
         0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  0]])
# Now we calculate the different scores
predictArray = arrayFewShotCoTNull
kappa = calculateAgreement(predictArray)
print("Kappa score: " + str(kappa))

# as row 4, 6 and 21 are used to give examples, those are removed for the F1 score.
numAnno = np.delete(numAnnotated, [4, 6, 21], axis=0)
f1RePr = calcF1RecallPrecision(numAnno, predictArray, 10)
display(f1RePr)

num = getNumLabels(predictArray, 10)
print(num)

frameZeroShot = pd.DataFrame(predictArray, columns= labelsList)
labels = frameZeroShot.apply(getLabels, axis=1)
pd.set_option('display.max_colwidth', None)
display(labels)

the next part contains the results for the per annotator fewshot method

In [ ]:
# We need other annotated arrays for that
annotatedDataFrameNoNaN1 = annotated1Clean.fillna(0)
annotatedDataFrameNoNaN2 = annotated2Clean.fillna(0)
annotatedDataFrameNoNaN3 = annotated3Clean.fillna(0)
annotatedDataFrameNoNaN4 = annotated4Clean.fillna(0)
annotatedDataFrameNoNaN5 = annotated5Clean.fillna(0)

# transform the annotated data to a numpy array to do the evaluations with
numAnnotated1 = pd.DataFrame.to_numpy(annotatedDataFrameNoNaN1)
numAnnotated2 = pd.DataFrame.to_numpy(annotatedDataFrameNoNaN2)
numAnnotated3 = pd.DataFrame.to_numpy(annotatedDataFrameNoNaN3)
numAnnotated4 = pd.DataFrame.to_numpy(annotatedDataFrameNoNaN4)
numAnnotated5 = pd.DataFrame.to_numpy(annotatedDataFrameNoNaN5)

# Now get only the actually annotated rows
numOnlyAnno1 = numAnnotated1[0:50]
numOnlyAnno2 = numAnnotated2[0:50]
numOnlyAnno3 = numAnnotated3[0:50]
numOnlyAnno4 = numAnnotated4[0:50]
numOnlyAnno5 = numAnnotated5[0:50]
# as row 4, 6 and 21 are used to give examples, they are removed for the evaluation
numOnlyAnno1 = np.delete(numOnlyAnno1, [4, 6, 21], axis=0)
numOnlyAnno2 = np.delete(numOnlyAnno2, [4, 6, 21], axis=0)
numOnlyAnno3 = np.delete(numOnlyAnno3, [4, 6, 21], axis=0)
numOnlyAnno4 = np.delete(numOnlyAnno4, [4, 6, 21], axis=0)
numOnlyAnno5 = np.delete(numOnlyAnno5, [4, 6, 21], axis=0)


Annotator 1

In [ ]:
arrayAnno1 = np.array([[0, 0, 0, 6, 1, 2, 1, 2, 0, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 8, 6, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 10, 0, 0, 5, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 10, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 2, 0, 2, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 3]
, [0, 0, 8, 2, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 8, 6, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 10, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 10, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 8, 3, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 10, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 10, 6, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1]
, [0, 0, 0, 0, 10, 9, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 1, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 10, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 10, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 10, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 5, 6, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 5, 5, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 10, 6, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 1, 3, 0, 1, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 4, 4, 0, 0, 0, 0, 0, 2, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 10, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 10, 3, 0, 3, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 10, 10, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 2, 8, 0, 0, 0, 0, 0, 0, 9, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 1, 3, 0, 0, 0, 0, 0, 2, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
, [0, 0, 0, 0, 10, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 10, 7, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 10, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 1, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 10, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 1]
, [0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 9, 9, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
, [1, 0, 0, 0, 9, 9, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 9, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 4, 0, 0, 0, 0, 0]
, [0, 0, 0, 3, 4, 6, 0, 1, 0, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]
, [0, 0, 0, 0, 10, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0]
, [7, 0, 0, 0, 10, 8, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 10, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0]
, [0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 10, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 7, 3, 5, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0]
, [0, 0, 0, 1, 1, 7, 0, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
, [0, 0, 0, 0, 10, 6, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 1, 0, 0, 0, 0, 0]
, [0, 0, 0, 3, 5, 6, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1]
, [0, 0, 0, 0, 10, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 1, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 5, 9, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0]
, [1, 0, 0, 0, 10, 0, 0, 0, 5, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0]])
# Now we calculate the different scores
predictArray = arrayAnno1
kappa = calculateAgreement(predictArray)
print("Kappa score: " + str(kappa))

f1RePr = calcF1RecallPrecision(numOnlyAnno1, predictArray, 10)
display(f1RePr)

num = getNumLabels(predictArray, 10)
print(num)

frameZeroShot = pd.DataFrame(predictArray, columns= labelsList)
labels = frameZeroShot.apply(getLabels, axis=1)
pd.set_option('display.max_colwidth', None)
display(labels)

Annotator 2

In [ ]:
arrayAnno2 = np.array([[0, 0, 0, 3, 1, 3, 3, 1, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4]
, [0, 0, 0, 6, 2, 2, 3, 0, 0, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 5, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 4]
, [0, 0, 0, 0, 10, 4, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 3, 1, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 4]
, [0, 0, 0, 8, 0, 0, 0, 0, 0, 4, 7, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 6, 2, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1]
, [1, 0, 0, 0, 8, 2, 0, 6, 5, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 4, 1, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 8, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3]
, [0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 8, 4, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]
, [0, 0, 0, 0, 5, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]
, [0, 0, 0, 0, 7, 4, 0, 8, 3, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 4, 0, 1, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 8, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 3]
, [0, 0, 0, 0, 7, 1, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2]
, [0, 0, 0, 0, 8, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0, 0, 0, 2]
, [0, 0, 3, 4, 0, 0, 0, 0, 0, 2, 6, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
, [0, 0, 0, 2, 1, 0, 3, 2, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1]
, [0, 0, 7, 5, 0, 0, 0, 0, 0, 4, 10, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 3, 0, 0, 0, 0, 0, 1, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]
, [0, 0, 2, 7, 0, 0, 0, 0, 0, 2, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
, [0, 0, 0, 0, 4, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 5]
, [0, 0, 0, 0, 7, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]
, [0, 0, 0, 0, 10, 2, 0, 1, 8, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 8, 7, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 1, 2, 0, 0, 0, 0, 0, 2, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 9, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 1]
, [0, 0, 0, 0, 6, 4, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]
, [0, 0, 0, 0, 10, 7, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 10, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1]
, [0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 6, 2, 0, 0, 0, 0, 3, 7, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
, [2, 0, 0, 0, 7, 6, 0, 1, 3, 0, 0, 0, 0, 1, 0, 1, 0, 2, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 2, 8, 6, 0, 2, 2, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
, [0, 0, 0, 4, 1, 3, 2, 1, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4]
, [0, 0, 0, 0, 6, 4, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0]
, [3, 0, 0, 0, 9, 3, 0, 7, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 5, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5]
, [0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
, [0, 0, 0, 0, 6, 6, 0, 1, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]
, [0, 0, 2, 9, 1, 1, 0, 0, 0, 2, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 7, 2, 0, 0, 0, 3, 5, 2, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
, [1, 0, 0, 0, 8, 4, 0, 4, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 6, 2, 3, 0, 0, 0, 2, 5, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
, [0, 0, 0, 0, 8, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5]
, [0, 0, 0, 0, 3, 2, 2, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 3]
, [4, 0, 0, 0, 6, 0, 0, 0, 3, 0, 0, 0, 0, 5, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]], )

# Now we calculate the different scores
predictArray = arrayAnno2
kappa = calculateAgreement(predictArray)
print("Kappa score: " + str(kappa))

f1RePr = calcF1RecallPrecision(numOnlyAnno2, predictArray, 10)
display(f1RePr)

num = getNumLabels(predictArray, 10)
print(num)

frameZeroShot = pd.DataFrame(predictArray, columns= labelsList)
labels = frameZeroShot.apply(getLabels, axis=1)
pd.set_option('display.max_colwidth', None)
display(labels)

Annotator 3

In [ ]:
arrayAnno3 = np.array([[0, 0, 0, 0, 0, 4, 4, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 0, 1, 2, 0, 0, 0, 1]
, [1, 0, 0, 0, 2, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 1, 5, 0, 0, 0, 0, 1]
, [0, 0, 0, 0, 7, 4, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 1, 0, 0, 0, 1]
, [0, 0, 0, 0, 7, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 3, 1, 1, 0, 0, 0, 0]
, [0, 0, 0, 0, 3, 1, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 1, 0, 0, 0, 0, 4]
, [0, 0, 3, 4, 1, 1, 0, 0, 0, 2, 5, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 7, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 4, 1, 1, 1, 0, 0, 0, 0]
, [0, 0, 0, 0, 5, 3, 0, 2, 0, 0, 0, 0, 0, 1, 0, 4, 0, 1, 0, 0, 3, 7, 4, 1, 0, 0, 0, 0]
, [0, 0, 0, 0, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 1, 0, 0, 0, 0, 4]
, [0, 0, 2, 4, 1, 1, 0, 0, 0, 2, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]
, [0, 0, 0, 0, 4, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 1, 0, 0, 0, 2]
, [0, 0, 0, 0, 5, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 7, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 1, 6, 3, 0, 0, 0, 0]
, [0, 0, 0, 0, 7, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 4, 0, 0, 0, 0, 2]
, [0, 0, 0, 0, 8, 5, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 2, 4, 0, 0, 0, 0]
, [0, 0, 0, 0, 10, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0]
, [0, 0, 6, 4, 0, 0, 0, 0, 0, 2, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
, [0, 0, 0, 1, 3, 2, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0]
, [0, 0, 2, 4, 3, 3, 0, 0, 0, 1, 5, 2, 0, 0, 2, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0]
, [0, 0, 0, 3, 1, 1, 2, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]
, [0, 0, 4, 6, 0, 0, 0, 0, 0, 1, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
, [0, 0, 0, 0, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 5, 2, 2, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 8, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2]
, [0, 0, 0, 0, 8, 5, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 5, 0, 2, 0, 0, 0, 0, 0]
, [0, 0, 2, 8, 0, 1, 0, 1, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 0, 1]
, [0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
, [0, 0, 0, 0, 8, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 4, 2, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 9, 4, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 8, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 10, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 2]
, [0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 4, 3, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 1, 5, 5, 0, 0, 0, 1, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 2, 1, 0, 0, 0, 1]
, [1, 0, 0, 0, 4, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 2, 0, 2, 1, 0, 5, 5, 1, 1, 0, 0, 0, 0]
, [0, 0, 0, 0, 6, 7, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2, 6, 2, 0, 0, 0, 1]
, [0, 0, 0, 2, 0, 2, 2, 2, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 4]
, [0, 0, 0, 0, 8, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 2, 3, 2, 0, 0, 0, 0]
, [0, 0, 0, 0, 9, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 7, 5, 1, 1, 0, 0, 0, 0]
, [0, 0, 0, 0, 9, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 2]
, [0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 6, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
, [0, 0, 0, 0, 10, 3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 2, 1, 3, 3, 0, 0, 0, 0]
, [0, 0, 1, 7, 0, 1, 1, 0, 0, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 3, 0, 0, 0, 1]
, [0, 0, 0, 2, 2, 3, 0, 3, 2, 2, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 4, 1, 1, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 10, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 5, 7, 3, 0, 0, 0, 1]
, [0, 0, 0, 4, 3, 3, 0, 1, 0, 3, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 3, 0, 0, 0, 2]
, [0, 0, 0, 0, 10, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0, 1, 0, 0, 0, 3]
, [0, 0, 0, 0, 2, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 5, 5, 0, 1, 0, 0, 0, 1]
, [1, 0, 0, 0, 6, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 6, 1, 0, 0, 0, 0, 0, 0]])

# Now we calculate the different scores
predictArray = arrayAnno3
kappa = calculateAgreement(predictArray)
print("Kappa score: " + str(kappa))

f1RePr = calcF1RecallPrecision(numOnlyAnno3, predictArray, 10)
display(f1RePr)

num = getNumLabels(predictArray, 10)
print(num)

frameZeroShot = pd.DataFrame(predictArray, columns= labelsList)
labels = frameZeroShot.apply(getLabels, axis=1)
pd.set_option('display.max_colwidth', None)
display(labels)

Annotator 4

In [ ]:
arrayAnno4 = np.array([[0, 0, 3, 5, 0, 0, 0, 0, 0, 3, 4, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 2, 0, 0, 1, 0, 0, 1]
, [0, 0, 2, 2, 3, 1, 0, 0, 1, 2, 1, 0, 0, 1, 2, 0, 0, 0, 0, 3, 1, 2, 0, 0, 2, 0, 0, 1]
, [0, 0, 0, 0, 6, 0, 0, 4, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 2]
, [0, 0, 0, 0, 8, 3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 6, 0, 3, 0, 0, 0, 1]
, [0, 0, 0, 1, 1, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 3, 0, 0, 0, 0, 5]
, [0, 0, 6, 3, 0, 0, 0, 0, 0, 1, 8, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0]
, [0, 0, 0, 1, 3, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2, 6, 1, 1, 0, 0, 0, 1]
, [2, 0, 1, 2, 5, 2, 0, 1, 1, 2, 0, 0, 0, 4, 1, 2, 0, 1, 0, 0, 2, 2, 0, 1, 0, 0, 0, 0]
, [0, 0, 0, 0, 9, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 1, 4, 0, 0, 0, 0]
, [0, 0, 9, 9, 0, 0, 0, 0, 0, 1, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 5, 5, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 1, 0, 0, 4, 1, 0, 0, 0, 0, 0, 3]
, [0, 0, 0, 0, 3, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 2]
, [1, 0, 1, 1, 8, 1, 0, 1, 0, 0, 1, 0, 0, 5, 0, 0, 0, 0, 0, 0, 6, 3, 2, 1, 0, 0, 0, 0]
, [0, 0, 0, 0, 5, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 1, 0, 0, 0, 4]
, [0, 0, 0, 0, 10, 2, 0, 1, 1, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 10, 3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 2, 3, 0, 0, 0, 0, 0, 1]
, [0, 0, 7, 6, 0, 0, 0, 0, 0, 0, 9, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0]
, [0, 0, 1, 2, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 2, 1, 0, 0, 0, 0, 1]
, [0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 7, 2, 0, 0, 5, 0, 0, 0, 0, 4, 1, 1, 0, 0, 2, 0, 0, 0]
, [0, 0, 6, 1, 0, 0, 0, 0, 0, 1, 5, 0, 0, 0, 3, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 6, 6, 0, 0, 0, 0, 0, 1, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 6, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 7, 0, 2, 0, 0, 0, 0]
, [1, 0, 0, 0, 6, 0, 0, 1, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 1]
, [0, 0, 0, 0, 7, 3, 0, 0, 1, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 6, 2, 0, 1, 0, 0, 0, 0]
, [0, 0, 8, 8, 0, 0, 0, 0, 0, 0, 7, 3, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]
, [0, 0, 5, 1, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
, [0, 0, 0, 0, 8, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 5, 2, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 6, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 1, 3, 0, 0, 0, 1]
, [0, 0, 1, 2, 8, 0, 0, 0, 0, 1, 2, 0, 0, 1, 0, 0, 0, 0, 0, 1, 2, 2, 3, 2, 0, 0, 0, 0]
, [0, 0, 0, 0, 9, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 5, 2, 1, 3, 0, 0, 0, 0]
, [0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 2, 0, 0, 0, 0, 6, 1, 0, 0, 0, 0, 0, 0]
, [0, 0, 3, 2, 1, 0, 0, 0, 0, 2, 3, 1, 0, 0, 2, 0, 0, 0, 0, 1, 1, 5, 0, 2, 0, 0, 0, 1]
, [0, 0, 0, 1, 7, 6, 0, 0, 2, 2, 1, 0, 0, 2, 0, 1, 0, 2, 1, 1, 1, 4, 0, 1, 0, 0, 0, 1]
, [0, 0, 5, 7, 3, 3, 0, 0, 1, 5, 3, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0]
, [0, 0, 0, 5, 1, 3, 1, 1, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 2, 1, 0, 0, 1]
, [0, 0, 2, 2, 3, 0, 0, 0, 0, 1, 2, 0, 0, 1, 0, 0, 0, 0, 0, 4, 2, 6, 0, 0, 0, 0, 0, 0]
, [1, 0, 0, 1, 5, 1, 0, 0, 1, 1, 1, 0, 0, 5, 0, 0, 0, 0, 0, 0, 3, 5, 2, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 6, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 1, 0, 3, 0, 0, 0, 1]
, [0, 0, 7, 7, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 7, 1, 0, 1, 0, 0, 0, 0, 0, 3, 0, 2, 0, 0, 0, 0, 2, 6, 0, 0, 0, 0, 0, 1]
, [0, 0, 6, 7, 0, 1, 0, 0, 0, 1, 7, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 1, 0, 0, 0, 0, 0]
, [0, 0, 8, 9, 0, 0, 0, 0, 0, 2, 6, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
, [1, 0, 0, 0, 5, 2, 0, 1, 1, 0, 0, 0, 0, 7, 0, 1, 0, 0, 0, 1, 6, 5, 0, 0, 0, 0, 0, 0]
, [0, 0, 4, 4, 1, 0, 0, 1, 0, 1, 3, 0, 0, 0, 1, 1, 0, 0, 0, 1, 3, 2, 0, 1, 1, 1, 0, 0]
, [0, 0, 0, 0, 8, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 2, 3, 0, 1, 0, 0, 0, 3]
, [0, 0, 0, 2, 2, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 3, 2, 0, 0, 0, 2]
, [2, 0, 0, 0, 7, 0, 0, 1, 1, 0, 0, 0, 0, 4, 0, 3, 0, 1, 0, 0, 2, 3, 0, 1, 0, 0, 0, 1]])

# Now we calculate the different scores
predictArray = arrayAnno4
kappa = calculateAgreement(predictArray)
print("Kappa score: " + str(kappa))

f1RePr = calcF1RecallPrecision(numOnlyAnno4, predictArray, 10)
display(f1RePr)

num = getNumLabels(predictArray, 10)
print(num)

frameZeroShot = pd.DataFrame(predictArray, columns= labelsList)
labels = frameZeroShot.apply(getLabels, axis=1)
pd.set_option('display.max_colwidth', None)
display(labels)

Annotator 5

In [ ]:
arrayAnno5 = np.array([[0, 0, 0, 2, 0, 0, 0, 1, 0, 2, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 6]
, [0, 0, 0, 0, 4, 5, 0, 5, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 1, 0, 2, 1, 0, 0, 0, 3]
, [0, 0, 0, 0, 8, 0, 0, 2, 1, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 2]
, [0, 0, 0, 0, 8, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2, 0, 0, 0, 1]
, [0, 0, 0, 0, 1, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 7]
, [0, 0, 5, 1, 0, 0, 0, 0, 0, 1, 8, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0]
, [0, 0, 0, 0, 6, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 3, 0, 1, 2, 0, 0, 0, 1]
, [0, 0, 0, 0, 6, 5, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 6, 1, 3, 2, 0, 0, 0, 2]
, [0, 0, 0, 0, 8, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 2, 0, 0, 0, 3]
, [0, 0, 0, 8, 0, 0, 0, 0, 0, 5, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
, [0, 0, 0, 0, 9, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 2, 0, 0, 0, 1]
, [0, 0, 0, 0, 7, 1, 0, 2, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
, [0, 0, 0, 0, 10, 6, 0, 4, 1, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 2, 2, 3, 1, 0, 0, 0, 0]
, [0, 0, 0, 0, 10, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 1]
, [0, 0, 0, 0, 9, 3, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 5, 0, 1, 1, 0, 0, 0, 1]
, [0, 0, 0, 0, 9, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 3, 6, 0, 0, 0, 0, 0, 3, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0]
, [0, 0, 0, 0, 2, 2, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1]
, [0, 0, 1, 0, 2, 2, 0, 0, 0, 3, 5, 2, 0, 0, 1, 1, 0, 0, 0, 0, 2, 2, 2, 1, 1, 0, 0, 0]
, [0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 6, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]
, [0, 0, 3, 3, 0, 0, 0, 0, 0, 2, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 8, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 2]
, [0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]
, [0, 0, 0, 0, 10, 7, 0, 2, 4, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 2, 6, 0, 0, 0, 0, 0, 4, 9, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]
, [0, 0, 1, 2, 0, 0, 0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]
, [0, 0, 0, 0, 9, 3, 0, 0, 1, 0, 0, 0, 0, 0, 0, 4, 0, 2, 0, 0, 4, 0, 1, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 5]
, [0, 0, 0, 0, 10, 2, 0, 0, 0, 1, 1, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 1, 1, 3, 0, 0, 0, 2]
, [0, 0, 0, 0, 9, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1]
, [0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 1, 4, 4, 2, 0, 0, 1, 1, 4, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2]
, [1, 0, 0, 0, 8, 7, 0, 1, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 3, 2, 1, 3, 0, 0, 0, 1]
, [0, 0, 0, 0, 6, 7, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 2, 2, 2, 4, 1, 0, 0, 0, 1]
, [0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 7]
, [0, 0, 0, 0, 10, 2, 0, 0, 2, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 2, 2, 2, 3, 2, 0, 0, 0, 0]
, [2, 0, 0, 0, 6, 5, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 2, 0, 3, 2, 0, 0, 0, 3]
, [0, 0, 0, 0, 9, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 1, 0, 0, 0, 1]
, [0, 0, 0, 4, 0, 0, 0, 0, 0, 3, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
, [0, 0, 0, 0, 8, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 2, 0, 0, 5, 0, 0, 2, 0, 0, 0, 1]
, [0, 0, 0, 7, 1, 1, 0, 0, 0, 4, 5, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 3]
, [0, 0, 1, 3, 1, 1, 0, 3, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 1, 0, 0, 0, 3]
, [1, 0, 0, 0, 9, 2, 0, 4, 2, 0, 0, 0, 0, 1, 0, 3, 0, 1, 0, 0, 6, 1, 0, 0, 0, 0, 0, 1]
, [0, 0, 0, 2, 4, 2, 0, 0, 1, 3, 2, 0, 0, 0, 0, 3, 0, 0, 0, 0, 2, 1, 0, 1, 0, 0, 0, 2]
, [0, 0, 0, 0, 8, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 3]
, [0, 0, 0, 0, 1, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 0, 6]
, [1, 0, 0, 0, 6, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 4, 3, 0, 0, 0, 0, 0, 2]])

# Now we calculate the different scores
predictArray = arrayAnno5
kappa = calculateAgreement(predictArray)
print("Kappa score: " + str(kappa))

f1RePr = calcF1RecallPrecision(numOnlyAnno5, predictArray, 10)
display(f1RePr)

num = getNumLabels(predictArray, 10)
print(num)

frameZeroShot = pd.DataFrame(predictArray, columns= labelsList)
labels = frameZeroShot.apply(getLabels, axis=1)
pd.set_option('display.max_colwidth', None)
display(labels)